In [16]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [17]:
data_original = pd.read_csv('Train.csv')
data_original.head(3)

,id,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,...,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,1,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,...,14/03/15,14/03/15,414,414,1803,PURCHASE,0.0,False,False,False
1,2,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,...,14/03/15,14/03/15,486,486,767,PURCHASE,0.0,True,False,False
2,3,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,...,14/03/15,14/03/15,486,486,767,PURCHASE,0.0,False,False,False


#### Escoger columnas

In [19]:
data_original.columns

Index(['id', 'accountNumber', 'customerId', 'creditLimit', 'availableMoney',
       'transactionDateTime', 'transactionAmount', 'merchantName',
       'acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardCVV', 'enteredCVV', 'cardLast4Digits',
       'transactionType', 'currentBalance', 'cardPresent',
       'expirationDateKeyInMatch', 'isFraud'],
      dtype='object')

In [20]:
data = data_original.drop(['id', 'accountNumber', 'customerId',
       'transactionDateTime', 'transactionAmount', 'merchantName',
       'acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardLast4Digits',
       'transactionType', 'cardPresent',
       'expirationDateKeyInMatch'], axis=1)

#### Construir modelo

In [21]:
data.head(3)

,creditLimit,availableMoney,cardCVV,enteredCVV,currentBalance,isFraud
0,5000,5000.0,414,414,0.0,False
1,5000,5000.0,486,486,0.0,False
2,5000,5000.0,486,486,0.0,False


In [24]:
data["isFraud"] = data["isFraud"].astype(int)

In [25]:
data.head(3)

,creditLimit,availableMoney,cardCVV,enteredCVV,currentBalance,isFraud
0,5000,5000.0,414,414,0.0,0
1,5000,5000.0,486,486,0.0,0
2,5000,5000.0,486,486,0.0,0


In [26]:
from sklearn.model_selection import train_test_split

X = data.drop(['isFraud'], axis=1)
y = data.isFraud

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=42)

In [27]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

model = Lasso(alpha=0.1)

_ = model.fit(X_train, y_train)

In [28]:
preds = model.predict(X_valid)
mean_absolute_error(y_valid, preds)

0.03100997011675692

In [64]:
preds

array([0.0173093 , 0.01408604, 0.01465787, ..., 0.01832185, 0.01694471,
       0.01659932])

In [30]:
preds.shape

(165136,)

### Generar solutions.csv

In [50]:
data_test1 = pd.read_csv('Test.csv')

#### Eliminar columnas

In [51]:
data_test = data_test1.drop(['accountNumber', 'customerId',
       'transactionDateTime', 'transactionAmount', 'merchantName',
       'acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardLast4Digits',
       'transactionType', 'cardPresent',
       'expirationDateKeyInMatch'], axis=1)

#### Validar modelo

In [52]:
data_test.head(3)

,id,creditLimit,availableMoney,cardCVV,enteredCVV,currentBalance
0,550453,250,211.81,451,451,38.19
1,550454,250,211.81,451,451,38.19
2,550455,250,207.67,451,451,42.33


In [53]:
ids_solution = data_test.id
data_test.drop('id', axis=1, inplace=True)
data_test.head(3)

,creditLimit,availableMoney,cardCVV,enteredCVV,currentBalance
0,250,211.81,451,451,38.19
1,250,211.81,451,451,38.19
2,250,207.67,451,451,42.33


In [54]:
ids_solution.shape

(235911,)

In [55]:
preds_solution = model.predict(data_test)

In [56]:
preds_solution.shape

(235911,)

In [58]:
solution_array = np.array([ids_solution,preds_solution]).reshape(len(ids_solution),2)

In [62]:
solution = pd.DataFrame(solution_array, columns = ['id','isFraud'])
solution.head(3)

,id,isFraud
0,550453.0,550454.0
1,550455.0,550456.0
2,550457.0,550458.0


In [61]:
solution.to_csv('solutions.csv')